# Numerical experiments in ergodic theory

## Why we call this Numerical Experiments

These are called numerical experiments, since, due to the finite nature of floating point arithmetic, the behavior of the discretized map and of the abstract dynamical system may be different.

There are many articles that deal with this topic, as:
- [Lanford, Oscar E., III Informal remarks on the orbit structure of discrete approximations to chaotic maps](https://people.math.harvard.edu/~knill/history/lanford/papers/LanfordOrbit.pdf)
- [Guineheuf-Monge Discrepancy and discretizations of circle expanding maps I: theory](https://arxiv.org/abs/2206.07991)
- [Guineheuf-Monge Discrepancy and discretizations of circle expanding maps II: simulations](https://arxiv.org/abs/2206.08000)
- [Galatolo-N-Rojas Probability, statistics and computation in dynamical systems](https://www.cambridge.org/core/journals/mathematical-structures-in-computer-science/article/abs/probability-statistics-and-computation-in-dynamical-systems/E20AF291F13006D356DD854F500A1853)

At the end, we will present an example, the BZ map from
- [Matsumoto-Tsuda, Noise Induced Order](https://link.springer.com/article/10.1007/BF01010923)
the first known example of Noise Induced Order.

## Birkhoff sums

We will implement now some numerical experiments dealing with computing finite time numerical approximation of Birkhoff averages, i.e., given an observable $\phi$, an initial condition $x_0$, and a time $N$ we compute
$$
A_N \phi(x_0) = \frac{1}{N}\sum_{i=0}^{N-1} \phi(T^i(x_0))
$$

In [ ]:
function orbit(f, x, n)
    v = Array{typeof(x), 1}(undef, n) #this declares an uninitialized vector
    v[1] = x
    for i in 2:n
        x = f(x)
        v[i] = x
    end
    return v
end

function orbit(f, x::Vector{T}, n) where {T} # this where T means that this is a parametric type
    k = length(x)
    v = Array{T, 2}(undef, (n, k)) 
    v[1, :] = x
    for i in 2:n
        x = f.(x)
        v[i, :] = x
    end
    return v
end

In [ ]:
f(x) = 4*x*(1-x)

The following is an histogram plot for the distribution of first $2000$ iterates of the orbit of the point $0.1$
under the action of the dynamic, compared to the invariant density of the map, which is known explictly 
to be
$$
f(x)=\frac{1}{\pi\sqrt{x(1-x)}}
$$
    

In [ ]:
v2000 = orbit(f, BigFloat(0.1), 2000)

In [ ]:
using Plots

In [ ]:
histogram(v2000, bins = 20, normalize = true, label = "Histogram plot", color = :orange)
plot!(x-> 1/(π*sqrt(x*(1-x))), color = :red, 0.005 , 0.995, linewidth = 3, label = "Invariant density" )

The following animation shows how $1000$ uniformly distributed initial points distribute themselves under iteration of the map $f$. Doing animations is quite simple, what we do is to make a for cycle that generates plots, and use a Julia macro to decorate the code.

In [ ]:
@doc animate

In [ ]:
v = orbit(f, 0.5 .+ 0.01*rand(100), 200)

In [ ]:
anim =  @animate for i in 1:200
    histogram(v[i, :], bins = 40, label = "$i", normalize = true, xlims = (0, 1))
end 
gif(anim, "tutorial_anim_fps30.gif", fps = 5)

# Uniformly expanding maps

We change the dynamic now into something uniformly expanding.

In [ ]:
g(x) = mod(2*x+0.1*sin(2*pi*x), 1)

In [ ]:
plot(g, 0, 1)

In [ ]:
v = orbit(g, rand(Float64, 1000), 400)

In [ ]:
anim =  @animate for i in 1:400
    histogram(v[i, :], bins = 40, label = "$i", normalize = true, ylims = (0, 1.5) )
end 
gif(anim, "tutorial_anim_fps30.gif", fps = 5)

So, up to know we have a way to compute the orbits, we want to compute some Birkhoff averages of some observables.
Remark that the code I'm writing is storing a lot of objects in memory and can be made more efficient.

The empirical Birkhoff average along a orbit of length $N$ is
$$
\frac{1}{N} \sum_{i=0}^{N-1}\phi(f^i(x))
$$
we will compute the Birkhoff averages for $1000$ starting points along orbits of length $400$.

In [ ]:
ϕ(x) = sin(x)+x^2 #ϕ is written by writing \phi and pressing tab

In [ ]:
function BirkhoffAverages(ϕ, v)
    w = ϕ.(v) # we evaluate the observables on the orbit
    k, n = size(w)
    z = accumulate(+, w; dims = 1) # this computes the accumulated sums along columns 
    t = [x for x in 1:k]
    z = z./t # we divide the first line by 1, the second by 2, the third by 3, etc...
    return z
end 

In [ ]:
z = BirkhoffAverages(ϕ, v)

In [ ]:
anim = @animate for i in 1:400
    histogram(z[i, :], bins = 40, label = "$i", xlims = (0.5, 1.1), ylims = (0, 10), normalize = true)
end 
gif(anim, "BirkhoffAverages.gif", fps = 5)

This animations shows the CLT in action for uniformly expanding maps. As $N$ grows the Birkhoff averages distribute as a Gaussian with average the Birkhoff average of the observable.

### Automatic Differentiation and computing the Lyapunov exponent

The package DualNumbers permits us to compute automatically the derivatives of a function (Automatic Derivation, AD for short).

In [ ]:
using Pkg;
Pkg.add("DualNumbers") # You will need to run this only once, to install the package

In [ ]:
using DualNumbers # this brings the AD package into namespace

A dual number is a number that consists of two components, called value and epsilon; we can think of a dual number as the "jet" of a function, i.e.,

```Dual(0, 1.0)```

can be thought of $0+x$ in a neighborhood of $0$. 
The relation between derivatives become therefore operators between dual numbers.
$$(a+\epsilon a')+(b+\epsilon b') = a+b+\epsilon (a'+b')$$
$$(a+\epsilon a')\cdot (b+\epsilon b') = a\cdot b+\epsilon (a\cdot b'+a'\cdot  b)$$

Overloading the operators and the functions to work with Dual numbers allows us to compute automatically derivatives.

In [ ]:
x  = Dual(0.1, 1.0)

In [ ]:
sin(x)

In [ ]:
f(x)

As you can see, the epsilon part is nothing else than $\cos(0.1)=(\sin)'(0.1)$.
We will use this to define automatically $\log(|T'|)$.

In [ ]:
ψ(x, h) = log(abs(h(Dual(x, 1)).epsilon))

We will now specialize it to our dynamic.

In [ ]:
ϕ(x) = ψ(x, g)

In [ ]:
z = BirkhoffAverages(ϕ, v)

In [ ]:
anim = @animate for i in 1:400
    histogram(z[i, :], bins = 40, label = "$i", xlims = (0.5, 0.9), ylims = (0, 50), normalize = true)
end 
gif(anim, "BirkhoffAverages.gif", fps = 5)

### Systems with noise

We want now do similar computations for systems with additive noise, i.e., the iterated of the system
are
$$X_i = T(X_{i-1})+\theta$$
where $\theta$ is a uniformly distributed random variable in $[-\xi, \xi]$.

The system we are going to study is a family of one-dimensional systems on $[-1,1]$; please note that
the noise could bring us outside $[-1, 1]$ so we need to set boundary conditions.
We will set periodic boundary conditions.
The following function applys the boundary conditions.

In [ ]:
function BC(x)
    if x>1
        return 1-(x-1)
    elseif x<0 
        return abs(x)
    else 
        return x
    end
end

In [ ]:
plot(BC, -3, 3)

In [ ]:
function orbit_with_noise(f, x::Vector{T}, ξ, n) where {T} # this where T means that this is a parametric type
    k = length(x)
    v = Array{T, 2}(undef, (n, k)) 
    
    v[1, :] = x
    for i in 2:n
        noise = ξ*(2*rand(T, k).-1.0)
        #@info sum(noise)
        x = BC.(f.(x)+noise)
        v[i, :] = x
    end
    return v
end

We will now look at the dynamic of a classical map, the BZ-map.

In [ ]:
a_big = big"0.5060735690368223513195993710530479569801417368282037493809901142182256388277859"
b_big = big"0.02328852830307032054478158044023918735669943648088852646123182739831022528"
c_big = big"0.121205692738975111744666848150620569782497212127938371936404761693002104361"


function BZ(x; T = Float64, a = T(a_big, RoundNearest), b = T(b_big, RoundNearest), c = T(c_big, RoundNearest))
    if 0<=x<=1/8
        return (a-abs(x-1/8)^(1/3))*exp(-x)+b
    elseif 1/8<x<0.3
        return (a+abs(x-1/8)^(1/3))*exp(-x)+b
    else
        return c*(10*x*exp((-10/3)*x))^(19)+b
    end
end

In [ ]:
plot(BZ, 0, 1)

In [ ]:
ξ = 0.873*10^(-4)/2

v = orbit_with_noise(BZ, rand(Float64, 200), ξ, 100000);

In [ ]:
histogram(v[end, :], bins = 40)

In [ ]:
ϕ(x) = ψ(x, BZ) # log(|BZ'|)

In [ ]:
z = BirkhoffAverages(ϕ, v)

In [ ]:
anim = @animate for i in 1:2000
    histogram(z[i, :], bins = 40, label = "$i", xlims = (-0.5, 0.5), ylims = (0, 20), normalize = true)
end 
gif(anim, "BirkhoffAverages.gif", fps = 30)

In [ ]:
ξ = 0.218*10^(-3)/2
v = orbit_with_noise(BZ, rand(Float64, 200), ξ, 100000)

In [ ]:
histogram(v[end, :], bins = 40)

In [ ]:
z = BirkhoffAverages(ϕ, v)

In [ ]:
sum(z[end, :])/200

In [ ]:
histogram(z[end, :], bins = 40)

# Final comments

In this notebook we saw many non-rigorous tools to investigate dynamical systems with Julia. We used histograms to see how orbits distribute in the interval $[0, 1]$ under the action of the dynamics.

We showed how to compute orbits, how to approximate the Birkhoff averages of observables and how to approximate an important quantity, the Lyapunov exponent (associated to the Physical measure).

We showed how this can be computed with various methods and different precisions, but all of these methods do not study the true dynamics, but a floating point approximation of it.
To show this, I will show one last example.

In [ ]:
doubling(x) = mod(2*x, 1)

In [ ]:
v = orbit(doubling, Float64(π)/4, 52)

In [ ]:
histogram(v, bins = 10, normalize = true, label = "10 iterates")

In [ ]:
v = orbit(doubling, Float64(π)/4, 500)

In [ ]:
histogram(v, bins = 10, normalize = true, label = "500 iterates")

In [ ]:
setprecision(1024)
v = orbit(doubling, BigFloat(π)/4, 500);
histogram(v, bins = 10, normalize = true, label = "500 iterates BigFloat")

The issue is that the computer represents numbers in base $2$ and the map $2x\, mod\, 1$ acts as a shift in base $2$; due to the fact that the the computer is a finite machine and pads with $0$, for any floating point number all the orbits converge to $0$ in a number of iterates equal to the precision. 

In [ ]:
setprecision(1024)
v = orbit(doubling, BigFloat(π)/4, 2000);
histogram(v, bins = 10, normalize = true, label = "2000 iterates BigFloat")

The number $\pi/4$ has infinite binary expansion, and a mathematical analysis of the orbit of $\pi/4$ under the action of the dynamics shows that it does not land on $0$ but its binary representation does.
In general, since we have only a finite number of floating point numbers, for any dynamics $f$ its floating point representation has only preperiodic orbits (by Pidgeon Hole Principle). 


This is a strong motivation for looking for other tools to understand the statistichal behaviour of a dynamical system, through the use of tools from Functional Analysis.

I will present them in the next days, starting tomorrow from Interval Arithmetics, a tool that allows us to do Validated numerical computations.